# FHIR Implementation Guide Testing Pipeline
This notebook provides a comprehensive pipeline for automatically extracting requirements from FHIR Implementation Guides and generating executable test suites. The pipeline transforms Implementation Guide (IG) documentation into structured test code that can validate FHIR server implementations.

## Overview
This automated pipeline takes FHIR Implementation Guide documentation and produces comprehensive test suites through several integrated stages:

- Implementation Guide Preparation: Convert and clean IG HTML documentation to markdown format
- Requirements Extraction: Use AI to identify and extract testable requirements from the IG
- Requirements Refinement: Consolidate and refine the extracted requirements
- Requirements Downselection: Combine multiple requirement sets and remove duplicates
- Test Plan Generation: Convert requirements into detailed test specifications
- Test Kit Generation: Generate executable Inferno test code

## Running this Notebook
The notebook is structured to run each stage sequentially. You can either:

- Run the complete pipeline: Execute all cells to process a complete IG
- Run individual stages: Execute specific sections as needed

Each stage will prompt you for inputs or use sensible defaults. The pipeline automatically saves intermediate outputs in checkpoint directories for review and iteration.

### Pipeline Stages
#### Stage 1: Implementation Guide Preparation
Text Extraction and Cleaning
- Converts HTML IG files to markdown format using Markdownify
- Cleans unnecessary content (navigation, headers, formatting artifacts)
- Prepares clean, structured text for AI processing
Inputs: HTML files from FHIR IG downloads
Outputs: Clean markdown files in checkpoints/text_extraction/ and checkpoints/post_processing/

#### Stage 2: Requirements Extraction
AI-Powered Requirements Identification
- Processes markdown files using AI to extract clear, testable requirements
- Formats requirements according to INCOSE Systems Engineering standards
- Generates structured requirements with IDs, descriptions, actors, and conformance levels
- Handles large documents through intelligent chunking
- Provides proper attribution and source tracking
Inputs: Cleaned markdown files
Outputs: Structured requirements list in checkpoints/requirements_extraction/ in markdown file

#### Stage 3: Requirements Refinement
AI-Based Requirements Consolidation
- Filters and identifies only testable requirements from raw extractions
- Consolidates duplicate requirements and merges related ones
- Applies consistent formatting and structure
- Removes non-testable assertions and narrative content
Inputs: Raw requirements from extraction stage in markdown file
Outputs: Refined requirements list in checkpoints/revised_reqs_extraction/

#### Stage 4: Requirements Downselection
Multi-Source Requirements Integration
- Combines multiple requirements lists from different extraction runs
- Uses semantic similarity analysis to identify and remove duplicates
- Creates a comprehensive, deduplicated final requirements set
- Provides statistical analysis of the consolidation process
Inputs: Multiple refined requirements files in markdown or JSON format
Outputs: Final consolidated requirements in checkpoints/requirements_downselect/ in markdown file

#### Stage 5: Test Plan Generation
Comprehensive Test Specification Development
- Transforms requirements into detailed test specifications
- Analyzes IG capability statements for context
- Generates implementation strategies with specific FHIR operations
- Creates structured test plans with validation criteria
Inputs: Refined requirements and IG capability statements in markdown format
Outputs: Detailed test plan in checkpoints/testplan_generation/ in markdown file

#### Stage 6: Test Kit Generation
Executable Test Code Creation
- Converts test specifications into executable Inferno Ruby tests
- Generates complete test suites with proper file organization
- Creates modular test structures following Inferno framework patterns
- Includes validation and alignment checking
Inputs: Test plan specification in markdown format
Outputs: Complete Inferno test kit in checkpoints/testkit_generation/

### Output Structure
The pipeline generates organized outputs in checkpoint directories:
checkpoints/
├── text_extraction/          # Converted markdown files
├── post_processing/          # Cleaned markdown files  
├── requirements_extraction/   # Initial AI-extracted requirements
├── revised_reqs_extraction/  # Refined requirements lists
├── requirements_downselect/  # Final consolidated requirements
├── testplan_generation/     # Detailed test specifications
└── testkit_generation/      # Executable Inferno test suites
Each stage preserves its outputs, allowing for iteration, review, and alternative processing paths.

## Setup

#### Importing Notebooks as Modules (from the [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/4.x/examples/Notebook/rstversions/Importing%20Notebooks.html))

In [1]:
import inspect
import json
import llm_utils
import importlib
import tiktoken
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from glob import glob

## Initializing LLM Clients

In [2]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [3]:
llm_clients.clients

{'claude': <anthropic.Anthropic at 0x105ee6b10>,
 'gemini': genai.GenerativeModel(
     model_name='models/gemini-2.5-pro',
     generation_config={'max_output_tokens': 8192, 'temperature': 0.3},
     safety_settings={<HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
     tools=None,
     system_instruction=None,
     cached_content=None
 ),
 'gpt': <openai.OpenAI at 0x111d4fc80>}

## Text Extraction

### HTML to Markdown Conversion

In [5]:
import html_narrative_extractor #import html extractor module

# Process directory with default settings
result = html_narrative_extractor.convert_local_html_to_markdown(
    "../us-core/test_set", #input directory
    "checkpoints/markdown1_test/" #output directory
)

Found 1 HTML files to process
Processed 1/1 files
Conversion complete. Successfully processed 1 files. Encountered 0 errors.


### Markdown Post-processing

In [6]:
import markdown_cleaner #import markdown cleaner module
markdown_cleaner.process_directory("checkpoints/markdown1_test", #input directory
                                   "checkpoints/markdown2_test/") #output directory

Found 1 markdown files in checkpoints/markdown1_test
Cleaned and saved: checkpoints/markdown2_test/CapabilityStatement-us-core-server.md

Processing complete: 1 files successfully cleaned, 0 failed


## Requirements Extraction

### Prompt-based Requirement Extraction

In [7]:
import reqs_extraction #import LLM requirements extraction module

INFO:root:Current working directory: /Users/ceadams/Documents/onclaive/onclaive/pipeline
INFO:root:Project root: /Users/ceadams/Documents/onclaive/onclaive
INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:root:Using prompts directory: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:root:Requirements extraction prompt: /Users/ceadams/Documents/onclaive/onclaive/prompts/requirements_extraction.md


In [8]:
reqs_extraction.run_requirements_extractor(
    'checkpoints/markdown2_test', #input directory
    'checkpoints/requirements_extraction/us-core', #output directory
    'claude', #LLM API of choice
    llm_clients)

INFO:root:Found markdown directory at checkpoints/markdown2_test
INFO:root:Found 1 markdown files
INFO:root:Processing with claude...
INFO:root:Starting processing with claude on directory: checkpoints/markdown2_test
INFO:root:Found 1 markdown files
INFO:root:Organized 1 files into 1 processing groups
INFO:root:Processing single file: CapabilityStatement-us-core-server.md
INFO:root:Split CapabilityStatement-us-core-server.md into 1 chunks using dynamic sizing
INFO:root:Processing chunk 1/1 of CapabilityStatement-us-core-server.md



Processing Implementation Guide with Claude...
This may take several minutes depending on the size of the Implementation Guide.


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Completed processing 1 files
INFO:root:Generated requirements document saved to checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250826_084443.md



Processing complete!
Generated requirements document: checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250826_084443.md
Processed 1 files


## Requirement Refinement with LLM

In [12]:
import reqs_reviewer
importlib.reload(reqs_reviewer)

<module 'reqs_reviewer' from '/Users/ceadams/Documents/onclaive/onclaive/pipeline/reqs_reviewer.py'>

### Large Number of Requirements (500+)

In [13]:
result = reqs_reviewer.run_batch_requirements_refinement(
    input_file="checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250826_084443.md",
    llm_client_instance=llm_clients,
    batch_size=50,
    api_type="claude"
)

INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts


STARTING BATCH PROCESSING
Input: checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250826_084443.md
Output: checkpoints/revised_reqs_extraction
Batch size: 50 requirements
API: claude

File size: 30,785 characters
Splitting requirements...
Found 67 total requirements
Will process in 2 batches

BATCH 1/2
   Requirements: 50 (#1-#50)
   Size: 23,375 chars (~5,843 tokens)


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


   Completed in 79.3s
   Pausing 2s...
   Progress: 1/2 (50.0%)
   ETA: 1.4 minutes remaining

BATCH 2/2
   Requirements: 17 (#51-#67)
   Size: 7,408 chars (~1,852 tokens)


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


   Completed in 16.1s
   Progress: 2/2 (100.0%)
   ETA: 0.0 minutes remaining

COMBINING RESULTS
--------------------
Merging batch results and renumbering...
   Processing batch 1 results...
   Processing batch 2 results...
   Renumbered 66 requirements
BATCH PROCESSING COMPLETE!
Output saved: checkpoints/revised_reqs_extraction/claude_refined_requirements_batch_20250826_085149.md
Original requirements: 67
Final requirements: 66
Successful batches: 2/2
Failed batches: 0/2
Total time: 1.6 minutes
Average per batch: 48.7 seconds


### Small Number of Requirements (<500)

In [14]:
# Manual refinement with specific file
refinement_result = reqs_reviewer.refine_requirements(
    input_file="checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250826_084443.md",
    api_type="claude",
    output_dir="checkpoints/revised_reqs_extraction",
    llm_client_instance=llm_clients
)

print(f"✅ Refined requirements saved to: {refinement_result['output_file']}")
print(f"📊 {refinement_result['original_requirements_count']} → {refinement_result['requirements_count']} requirements")

if refinement_result.get('warnings'):
    print("⚠️  Warnings:")
    for warning in refinement_result['warnings']:
        print(f"  - {warning}")

INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:root:Starting requirements refinement with claude
INFO:root:Original requirements count: 67
INFO:root:Input size: 30785 characters, ~7696 tokens
INFO:root:Sending requirements to claude for refinement...
INFO:root:Estimated input tokens: 8307
INFO:root:API input limit: 180000
INFO:root:API output limit: 8192
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Requirements refinement complete. Output saved to: checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250826_085415.md
INFO:root:Refined 67 -> 66 requirements
INFO:root:Output size: 29582 characters, ~7395 tokens


✅ Refined requirements saved to: checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250826_085415.md
📊 67 → 66 requirements


## Requirement Downselection
#NOTE: right now, output file will overwrite itself- need to update code to change name of output file

In [19]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=["checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250826_084443.md", "checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250826_085415.md"]
    #rag_files=["checkpoints/requirements_extraction/RAG/raw_output.json"]
    )

/Users/ceadams/Documents/onclaive/onclaive/pipeline/requirement_downselect.py:23: SyntaxWarning: invalid escape sequence '\*'
  headings = list(re.finditer("\*\*\w+\*\*\:", split))
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Final number of requirements: 62
Output saved in Markdown format in directory: checkpoints/requirements_downselect


## Test Plan Generation

In [17]:
import logging
llm_clients.logger.setLevel(logging.INFO)

##### Without RAG- Faster

In [ ]:
import req_to_testplan
importlib.reload(req_to_testplan)

result = req_to_testplan.generate_consolidated_test_plan(
    llm_clients,
    'claude',  # 'claude' or 'gemini' or 'gpt'
    llm_clients.logger,
    "checkpoints/requirements_downselect/filtered_requirements.md",
    "../us-core/test_set/CapabilityStatement-us-core-server.html",
    "US Core IG",
    output_dir='checkpoints/testplan_generation/us-core'
)

print(f"Generated test plan with improved capability parsing: {result['test_plan_path']}")

INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:llm_utils:Starting test plan generation with claude for US Core IG
INFO:llm_utils:Parsed 22 requirements from checkpoints/requirements_downselect/filtered_requirements.md
INFO:llm_utils:Parsed capability statement from ../us-core/test_set/CapabilityStatement-us-core-server.html
INFO:llm_utils:Identifying group for requirement REQ-01 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:llm_utils:Identifying group for requirement REQ-42 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:llm_utils:Identifying group for requirement REQ-43 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:llm_utils:Identifying group for requirement REQ-44 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 2

##### With RAG

In [ ]:
import warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set logging level to reduce noise
import logging
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
logging.getLogger("backoff").setLevel(logging.ERROR)

import req_to_testplan_rag
importlib.reload(req_to_testplan_rag)

#clearing any existing capability statements from vector database
req_to_testplan_rag.clear_capability_collection("capability_statements")

Deleted existing collection: capability_statements


ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


In [ ]:
req_to_testplan_rag.generate_consolidated_test_plan(
    llm_clients, 
    'claude', 
    llm_clients.logger, 
    "checkpoints/requirements_downselect/filtered_requirements.md", 
    "checkpoints/markdown2/CapabilityStatement-us-core-server.md", 
    "US Core IG",
    output_dir='checkpoints/testplan_generation/us-core',
    verbose=True)

INFO:llm_utils:Starting test plan generation with claude for Plan-Net IG
INFO:llm_utils:Parsed 19 requirements from /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md
INFO:llm_utils:Initialized capability collection from ../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
INFO:llm_utils:Identifying group for requirement REQ-01 using claude...
INFO:llm_utils:Identifying group for requirement REQ-02 using claude...
ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))
INFO:llm_utils:Identifying group for requirement REQ-03 using claude...
INFO:llm_utils:Identifying group for requirement REQ-04 using claude...
INFO:


Processing Group: General/Cross-Resource (15 requirements)

Processing REQ-03: Health Plan API Must Support data population capability

RAG RETRIEVAL FOR REQ-03
Query: Health Plan API Must Support data population capability "Health Plan API actors SHALL be capable of populating all Must Support data elements as part of the query results." Health Plan API FHIR SHALL
Searching for 5 most relevant capability chunks...

Found 5 matching chunks:

  Match 1 (distance: 0.7221184968948364):
  Length: 3036 chars
  Preview: FHIR Resource Detail: #### HealthcareService

Conformance Expectation: **SHALL**

Supported Profiles...
  ...lthcareService?_lastUpdated=[_lastUpdated]` |

---

  Match 2 (distance: 0.7407464981079102):
  Length: 3288 chars
  Preview: FHIR Resource/Component: ### RESTful Capabilities by Resource/Profile:

**Summary of Search Criteria...
  ...nerRole:network, PractitionerRole:endpoint |  |  |

  Match 3 (distance: 0.8931890726089478):
  Length: 3225 chars
  Preview: FHIR Reso

ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


Completed test specification for REQ-03


INFO:llm_utils:Processing requirement for group 'General/Cross-Resource': REQ-04
INFO:llm_utils:Generating test specification for REQ-04 using claude...


...

INFO:llm_utils:Processing requirement for group 'Security': REQ-18
INFO:llm_utils:Generating test specification for REQ-18 using claude...


Completed test specification for REQ-02

Processing Group: Security (1 requirements)

Processing REQ-18: Plan-Net Server unauthorized request rejection

RAG RETRIEVAL FOR REQ-18
Query: Plan-Net Server unauthorized request rejection "A server SHALL reject any unauthorized requests by returning an `HTTP 401` unauthorized response code." Plan-Net Server FHIR SHALL
Searching for 5 most relevant capability chunks...

Found 5 matching chunks:

  Match 1 (distance: 0.5878045558929443):
  Length: 1169 chars
  Preview: FHIR Resource/Component: ### FHIR RESTful Capabilities

The Plan\-Net Server **SHALL**:

1. Support ...
  ...eturning an `HTTP 401` unauthorized response code.

  Match 2 (distance: 0.7881736159324646):
  Length: 996 chars
  Preview: FHIR Major Section: ## CapabilityStatement: Plan\-Net CapabilityStatement

| *Official URL*: http://...
  ...openapi.json) \| [Download](plan-net.openapi.json)

  Match 3 (distance: 0.9208030700683594):
  Length: 802 chars
  Preview: FHIR Major Secti

INFO:llm_utils:Consolidated test plan saved to checkpoints/testplan_generation/claude_test_plan_20250730_140520.md


Completed test specification for REQ-18


{'requirements_count': 19,
 'group_count': 4,
 'test_plan_path': 'checkpoints/testplan_generation/claude_test_plan_20250730_140520.md'}

## Test Kit Generation

In [ ]:
import plan_to_tests
import importlib
importlib.reload(plan_to_tests)

plan_to_tests.generate_inferno_test_kit(
    llm_clients,
    'gpt',
    'checkpoints/testplan_generation/claude_test_plan_20250730_140520.md',
    #'../test_kit_dev/inferno-guidance.md', #not used right now
    output_dir='checkpoints/testkit_generation/us_core',
    expected_actors=["Server", "Client"] #not used right now
)

/Users/ceadams/Documents/onclaive/onclaive/pipeline/plan_to_tests.py:775: SyntaxWarning: invalid escape sequence '\A'
  """
INFO:plan_to_tests:Starting Inferno test generation with gpt for PlanNet
INFO:plan_to_tests:Parsed test plan into 4 sections
INFO:plan_to_tests:Found 19 total requirements
INFO:plan_to_tests:Loaded Inferno DSL guidance
INFO:plan_to_tests:Processing section: General/Cross-Resource with 15 requirements
INFO:plan_to_tests:Generating tests for section: General/Cross-Resource
INFO:plan_to_tests:Sending 122 tokens to gpt API (limit: 16000)
INFO:plan_to_tests:Attempting to generate tests for entire section: General/Cross-Resource


Found 19 potential requirements
Processing requirement: REQ-03
Added requirement REQ-03 to section General/Cross-Resource
Processing requirement: REQ-04
Added requirement REQ-04 to section General/Cross-Resource
Processing requirement: REQ-05
Added requirement REQ-05 to section General/Cross-Resource
Processing requirement: REQ-06
Added requirement REQ-06 to section General/Cross-Resource
Processing requirement: REQ-07
Added requirement REQ-07 to section General/Cross-Resource
Processing requirement: REQ-08
Added requirement REQ-08 to section General/Cross-Resource
Processing requirement: REQ-09
Added requirement REQ-09 to section General/Cross-Resource
Processing requirement: REQ-10
Added requirement REQ-10 to section General/Cross-Resource
Processing requirement: REQ-11
Added requirement REQ-11 to section General/Cross-Resource
Processing requirement: REQ-12
Added requirement REQ-12 to section General/Cross-Resource
Processing requirement: REQ-13
Added requirement REQ-13 to section G

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Generating test for requirement: REQ-03
INFO:plan_to_tests:Requirement REQ-03: Sending 1111 tokens to gpt API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement: REQ-03
INFO:plan_to_tests:Generating test for requirement: REQ-04
INFO:plan_to_tests:Requirement REQ-04: Sending 1135 tokens to gpt API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement: REQ-04
INFO:plan_to_tests:Generating test for requirement: REQ-05
INFO:plan_to_tests:Requirement REQ-05: Sending 1140 tokens to gpt API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement

{'total_sections': 4,
 'total_requirements': 19,
 'generated_tests': 19,
 'module_dir': 'checkpoints/testkit_generation/gpt_testkit_20250819_225027/plannet',
 'module_file': 'checkpoints/testkit_generation/gpt_testkit_20250819_225027/gpt_plannet_20250819_225027.rb',
 'output_dir': 'checkpoints/testkit_generation/gpt_testkit_20250819_225027',
 'timestamp': '20250819_225027'}